In [1]:
import pandas as pd

# Loading dataset
ind_df = pd.read_csv('data/indigenous_collection_processed.csv', index_col='id')
print(f'Dataframe columns: \n{ind_df.columns}')

Dataframe columns: 
Index(['url', 'thumbnail', 'creation_date', 'modification_date',
       'numero_do_item', 'tripticos', 'categoria', 'nome_do_item',
       'nome_do_item_dic', 'colecao', 'coletor', 'doador', 'modo_de_aquisicao',
       'data_de_aquisicao', 'ano_de_aquisicao', 'data_de_confeccao', 'autoria',
       'nome_etnico', 'descricao', 'dimensoes', 'funcao', 'materia_prima',
       'tecnica_confeccao', 'descritor_tematico', 'descritor_comum',
       'numero_de_pecas', 'itens_relacionados', 'responsavel_guarda',
       'inst_detentora', 'povo', 'autoidentificacao', 'lingua',
       'estado_de_origem', 'geolocalizacao', 'pais_de_origem', 'exposicao',
       'referencias', 'disponibilidade', 'qualificacao', 'historia_adm',
       'notas_gerais', 'observacao', 'conservacao', 'image_path'],
      dtype='object')


In [2]:
from IPython.core.magic import register_cell_magic

# Creating skip cell command
@register_cell_magic
def skip(line, cell):
    return

# Image Clustering

Clustering experiments with image feature extractors. The idea is to fine-tune some pre-trained models on our dataset and then remove the last layer of the model to cluster on the embedding space projections.

## Dataset Preparation

For fine-tuning the model on our dataset, we are going to try a few different labels and study how they affect the generated emebdding space. For now, we focus *povo* and *categoria*.

In [3]:
from PIL import Image

# Filtering out corrupted images
corrupted_images = []
for index, row in ind_df.loc[ind_df['image_path'].notna()].iterrows():
    try:
        Image.open(row['image_path'])
    except Exception as e:
        # print(e)
        corrupted_images.append(row['image_path'])
        ind_df.loc[index, 'image_path'] = pd.NA
print(f'{len(corrupted_images)} corrupted images')

# Creating 'image_path_br' column
ind_df['image_path_br'] = ind_df['image_path'].values
ind_df.loc[ind_df['image_path_br'].notna(), 'image_path_br'] = \
    ind_df.loc[ind_df['image_path_br'].notna(), \
               'image_path'].apply(lambda path: \
                                   f"data/br_images/{path.split('/')[-1].split('.')[0]}.png")

# Preparing labels for dataset training
label_column = 'povo' # 'categoria', 'povo', 'ano_de_aquisicao'
name_to_num = {c: i for i, c in enumerate(ind_df[label_column].unique())}
labels = {row['image_path_br']: name_to_num[row[label_column]] \
          for index, row in ind_df.loc[ind_df['image_path_br'].notna()].iterrows()}

1 corrupted images


In [4]:
import os
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Getting the proper device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Creating the ImageDataset class and the DataLoader object to avoid loading all the images
# simultaneously and run out of GPU memory
class ImageDataset(Dataset):
    def __init__(self, image_dir, labels, transform=None):
        self.image_dir = image_dir
        self.image_files = [f for f in os.listdir(image_dir) \
                            if f.endswith(('.png', '.jpg', '.jpeg'))]
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = self.labels.get(self.image_files[idx], -1)
        return image, label

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
dataset = ImageDataset("data/br_images/", labels, transform=transform)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True, num_workers=0, pin_memory=True)

## ViT Base Patch-16

### Pre-trained Embedding Space

In [5]:
# Projecting data onto the off-the-shelf pre-trained embedding space from ViT
from transformers import ViTImageProcessor, ViTModel
from tqdm import tqdm

# Loading model
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
model.to(device)

# Iterating over data
image_embeddings = []
for batch_images, _ in tqdm(dataloader):
    batch_images = batch_images.to(device)
    with torch.no_grad():
        outputs = model(batch_images)
    
    # Do I get the last_hidden_state of CLS token or the pooler_output?
    embeddings = outputs['last_hidden_state'][:, 0, :]
    # embeddings = outputs['pooler_output']
    image_embeddings.append(embeddings.cpu())

    # # Freeing space
    # del batch_images, outputs, embeddings
    # torch.cuda.empty_cache()

image_embeddings = torch.cat(image_embeddings, dim=0)
image_embeddings.size()

100%|██████████████████████████| 89/89 [02:39<00:00,  1.80s/it]


torch.Size([11274, 768])

In [6]:
# Visualizing data projection


### Fine-tuning Embedding Space